## Inversion Recovery Example

In this example, we will perform an inversion-recovery experiment on a Pyridine molecule. This serves as an excellent demonstration of the functionality provided by the Spinguin package.

- **Molecule and Spins**: The simulation incorporates all NMR-active spins of the Pyridine molecule, including:
    - All 5 protons.
    - The quadrupolar nitrogen-14 nucleus.

- **Performance Optimization**: The computational performance is enhanced by utilizing the restricted Liouville space.

- **Interactions Included**:
    - **Coherent Interactions**:
        - Zeeman interaction.
        - Chemical shift.
        - Scalar $J$-coupling.
    - **Incoherent Interactions**:
        - Dipolar coupling.
        - Chemical shift anisotropy.
        - Quadrupolar coupling.
        - Scalar relaxation of the second kind.

This example provides a comprehensive overview of how to simulate NMR experiments using Spinguin.

### Imports

To begin, we import all the necessary modules.

- **NumPy**: Used for creating arrays required for input parameters and storing results.
- **Matplotlib**: Utilized for plotting the simulation results.
- **Spinguin**: Provides specialized functionality for simulating NMR experiments.

In [ ]:
# Import necessary modules:
import numpy as np
import matplotlib.pyplot as plt
import spinguin as sg

### Simulation Parameters

The following experimental conditions are defined:

- **Magnetic Field (T)**: The strength of the static magnetic field $B_0$.
- **Temperature (K)**: The temperature at which the simulation is conducted.

The following acquisition parameters are defined:
- **Dwell Time (s)**: The interval between successive time points in the simulation.
- **Number of Points**: The total number of time steps for the simulation.
- **Isotope**: The isotope to be measured.

In [ ]:
# Define experimental conditions:
sg.parameters.magnetic_field = 1
sg.parameters.temperature = 295

# Define acquisition parameters
sg.parameters.dwell_time = 2e-3
sg.parameters.npoints = 50000
sg.parameters.isotope = "1H"

### Define the Spin System

Next, we perform the following:

- **Initialize Spin System**: We create a spin system that consists of 5 protons (1H) and 1 nitrogen (14N).
- **Define Maximum Spin Order**: To speed up the calculations, the maximum spin order is restricted to include at most 3 spins.
- **Build the Basis**: We build the Liouville space basis set for the spin system.

In [ ]:
# Define the spin system
spin_system = sg.spin_system(['1H', '1H', '1H', '1H', '1H', '14N'])
spin_system.basis.max_spin_order = 3
spin_system.basis.build()

### Define the Spin System Properties

To define the spin system, we need to specify the following parameters:

- **Chemical Shifts (ppm)**: The chemical shifts for each nucleus.
- **J Couplings (Hz)**: The $J$-coupling constants between nuclei.
- **XYZ Coordinates (Å)**: The spatial positions of the nuclei in Cartesian coordinates.
- **Shielding Tensors (ppm)**: The nuclear shielding tensors for each nucleus.
- **EFG Tensors (a.u.)**: The electric field gradient tensors for quadrupolar nuclei.

These parameters are written as NumPy arrays and assigned to the spin system attributes.

In [ ]:
# Define the spin system parameters:
spin_system.chemical_shifts = np.array([8.56, 8.56, 7.47, 7.47, 7.88, 95.94])

spin_system.J_couplings = np.array([
    [ 0,     0,      0,      0,      0,      0],
    [-1.04,  0,      0,      0,      0,      0],
    [ 4.85,  1.05,   0,      0,      0,      0],
    [ 1.05,  4.85,   0.71,   0,      0,      0],
    [ 1.24,  1.24,   7.55,   7.55,   0,      0],
    [ 8.16,  8.16,   0.87,   0.87,  -0.19,   0]
])

spin_system.xyz = np.array([
    [ 2.0495335,  0.0000000, -1.4916842],
    [-2.0495335,  0.0000000, -1.4916842],
    [ 2.1458878,  0.0000000,  0.9846086],
    [-2.1458878,  0.0000000,  0.9846086],
    [ 0.0000000,  0.0000000,  2.2681296],
    [ 0.0000000,  0.0000000, -1.5987077]
])

spin_system.shielding = np.zeros((6, 3, 3))
spin_system.shielding[5] = np.array([
    [-406.20,  0.00,    0.00],
    [   0.00, 299.44,   0.00],
    [   0.00,   0.00, -181.07]
])

spin_system.efg = np.zeros((6, 3, 3))
spin_system.efg[5] = np.array([
    [ 0.3069,  0.0000,  0.0000],
    [ 0.0000,  0.7969,  0.0000],
    [ 0.0000,  0.0000, -1.1037]
])

### Specify the Relaxation Theory

We will treat the relaxation using Redfield theory. The following parameters are set:

- **Theory**: We specify that Redfield theory is used.
- **SR2K**: We specify that scalar relaxation of the second kind is considered.
- **Thermalization**: Thermalization of the relaxation superoperator (Levitt-di Bari) is enabled.
- **Rotational Correlation Time (ps)**: The characteristic time for molecular tumbling, affecting relaxation processes.

In [ ]:
# Specify the Relaxation Theory
spin_system.relaxation.theory = "redfield"
spin_system.relaxation.tau_c = 10e-12
spin_system.relaxation.thermalization = True
spin_system.relaxation.sr2k = True

### Perform the Inversion Recovery Experiment

We obtain the evolution of the $z$-magnetization by performing the inversion-recovery experiment.

In [ ]:
magnetizations = sg.inversion_recovery(spin_system)

### Plot the Results

Congratulations! The simulation is complete. The results are stored in the `magnetizations` array. You can now:

- Save the results as a CSV file for further analysis.
- Perform additional analysis directly on the results.
- Plot and visualize the results.

In this example, we will plot and display the results.

In [ ]:
# Construct the time array
time = np.linspace(0, (sg.parameters.npoints[0]-1)*sg.parameters.dwell_time[0], sg.parameters.npoints[0])

# Plot the magnetization of each proton (spins with indices 0 to 4) over time.
for mag in magnetizations:
    plt.plot(time, np.real(mag), label=f"1H")

# Add a legend to identify each spin's magnetization curve.
plt.legend(loc="upper right")

# Label the x-axis and y-axis for clarity.
plt.xlabel("Time (s)")
plt.ylabel("Magnetization")

# Add a title to describe the plot.
plt.title("Inversion-Recovery of Pyridine")

# Adjust the layout to prevent overlapping of labels and elements.
plt.tight_layout()

# Display the plot.
plt.show()

# Clear the figure to avoid overlapping plots in subsequent cells.
plt.clf()